In [ ]:
# mount it
from google.colab import drive
drive.mount('/content/drive')

In [4]:
#!wget https://gist.githubusercontent.com/Yegorov/dc61c42aa4e89e139cd8248f59af6b3e/raw/20ac954e202fe6a038c2b4bb476703c02fe0df87/ya.py
#!chmod +x ya.py
!python ya.py https://disk.yandex.ru/d/481aq0zVzdo6iw ./

Traceback (most recent call last):
  File "/home/zorkin/Yandex.Disk/Обучение/Hackathon/omsk_chempionat/model-train/ya.py", line 19, in <module>
    json_res = json.loads(res)
  File "/home/zorkin/.pyenv/versions/3.10.0/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/home/zorkin/.pyenv/versions/3.10.0/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/home/zorkin/.pyenv/versions/3.10.0/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [ ]:
!unzip train_dataset_train.zip

In [ ]:
!rm train_dataset_train.zip

In [5]:
!mkdir train/images train/labels augumentation augumentation/images augumentation/labels augumentation/images/train augumentation/images/test augumentation/labels/train augumentation/labels/test

In [ ]:
!ls train

In [7]:
import json
from pathlib import Path

import albumentations as A
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
# width or x - горизонталь
# height or y - вертикаль
# Each row is - class x_center y_center width height format.
# Box coordinates must be in normalized xywh format (from 0 - 1). If your boxes are in pixels, divide x_center and width by image width, and y_center and height by image height.
# Class numbers are zero-indexed (start from 0).
# пример строки в файле .txt - '45 0.479492 0.688771 0.955609 0.5955\n'

In [8]:
def get_files(files_dir: Path, suff: str = 'txt'):
    return sorted(Path(files_dir).glob(f'*.{suff}'))


# Преобразование координат объектов в формат yolo

## Загрузка координат в pandas

In [9]:
location_csv = Path('train.csv')
location_image = Path('train/')
location_labels = Path('train/labels')

In [10]:
location = pd.read_csv(location_csv)

In [11]:
location.insert(3, 'short_name', ['']*location.shape[0])

In [12]:
def insert_short_name(x):    
    x['short_name'] = str(x['ID_img']).split('.')[0]
    return x

In [13]:
location = location.apply(insert_short_name, axis=1)

In [14]:
location.head(3)

,ID_img,count_region,region_shape,short_name
0,3436.JPG,0.0,0.0,3436
1,3437.JPG,0.0,0.0,3437
2,3438.JPG,0.0,0.0,3438


In [15]:
location[location['region_shape'] != '0.0'].head(3)

,ID_img,count_region,region_shape,short_name
8,3444.jpg,1.0,"['{""cx"":2259,""cy"":391,""r"":64}']",3444
217,3653.JPG,1.0,"['{""cx"":2719,""cy"":2097,""r"":75}']",3653
254,3690.JPG,1.0,"['{""cx"":2914,""cy"":1693,""r"":90}']",3690


In [16]:
def get_size(image_file: Path):
    forest_image = Image.open(image_file)
    width, height = forest_image.size
    return width, height

In [17]:
def get_fraction(image_size, location):
    # location consist: 
    # ‘r’ - область внутри которой находиться человек
    # ‘cx’ - центр окружности по координате x
    # ‘cy’ - центр окружности по координате y
    boxes = []
    width, height = image_size
    for box in location:        
        center_x = str(box['cx'] / width)
        center_y = str(box['cy'] / height)
        box_width = str(box['r'] / width)
        box_height = str(box['r'] / height)
        boxes.append(['0', center_x, center_y, box_width, box_height])
    return boxes


In [18]:
def get_text(boxes):
    boxes_str = []
    for location in boxes:
        new_line = ' '.join(location) + '\n'
        boxes_str.append(new_line)
    return boxes_str

In [19]:
def write_txt(boxes, name):
    txt_file = location_labels / f'{name}.txt'
    with open(txt_file, 'w') as new_file:
        for row in boxes:
            new_file.write(row)

In [20]:
def row_handler(row: pd.Series) -> None:
    #print(type(row))
    name, coord = row['ID_img'], row['region_shape']
    short_name = name.split('.')[0]
    boxes = json.loads(coord.replace("\'", ""))
    image_file = location_image / name
    size = get_size(image_file)
    boxes = get_fraction(size, boxes)
    boxes = get_text(boxes)
    write_txt(boxes, short_name)
    #print(boxes)

image_file = Path('../train_dataset_train/train/3771.JPG')
size = get_size(image_file)
size

coord = location.iloc[395]['region_shape']
boxes = json.loads(coord.replace("\'", ""))
boxes = get_fraction(size, boxes)
boxes = get_text(boxes)
boxes

In [21]:
# исключаем строки где нет людей и преобразуем координаты в текстовый файл
short_df = location[location['region_shape'] != '0.0'].apply(row_handler, axis=1)
short_df.head(3)

8      None
217    None
254    None
dtype: object

In [22]:
!ls train/labels

3444.txt  4465.txt  5277.txt  6153.txt	6714.txt  7078.txt  8048.txt  8464.txt
3653.txt  4604.txt  5314.txt  6219.txt	6741.txt  7200.txt  8103.txt  8507.txt
3690.txt  4780.txt  5478.txt  6221.txt	6854.txt  7263.txt  8147.txt
3835.txt  4871.txt  5680.txt  6233.txt	6906.txt  7577.txt  8155.txt
3947.txt  4888.txt  5824.txt  6368.txt	6908.txt  7713.txt  8218.txt
4153.txt  4934.txt  5956.txt  6478.txt	6911.txt  7722.txt  8257.txt
4269.txt  5065.txt  6105.txt  6561.txt	7012.txt  7872.txt  8380.txt
4291.txt  5191.txt  6146.txt  6684.txt	7049.txt  8020.txt  8406.txt


# Аугументация датасета

In [23]:
aug_dir = Path('augumentation')
aug_size = 256
person_num = 35
area_num = 30

In [24]:
labels_list = get_files(location_labels)

In [25]:
def transformer_person(image, bboxes, class_labels, size):
    width, height = size
    transform_person = A.Compose(
        [A.RandomSizedBBoxSafeCrop(width=width, height=height, erosion_rate=0.3)],
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']),
    )
    transformed = transform_person(image=image, bboxes=bboxes, class_labels=class_labels)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    transformed_class_labels = transformed['class_labels']    
    return transformed_image, transformed_bboxes

In [26]:
def transformer_area(image, bboxes, class_labels, size):
    width, height = size
    transform_area = A.Compose([
        A.RandomCrop(width=aug_size, height=aug_size),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
    ], bbox_params=A.BboxParams(format='yolo', min_visibility=0.2, label_fields=['class_labels']))
    transformed = transform_area(image=image, bboxes=bboxes, class_labels=class_labels)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    transformed_class_labels = transformed['class_labels']    
    return transformed_image, transformed_bboxes

In [27]:
def read_boxes(label_path: Path):
    with open(label_path, 'r') as label:
        label_list = label.readlines()
    return label_list

In [28]:
def get_image(image_path: Path):
    #image = Image.open(image_path)
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [29]:
def get_boxes(label_path: Path):
    boxes = read_boxes(label_path)
    boxes_list = []
    labels = []
    for box in boxes:
        box = box.replace('\n', '')
        box = box.split(' ')
        # срезаем до 8 знаков после запятой
        bbox = [float(x[:11]) for x in box[1:]]
        boxes_list.append(bbox)
        labels.append(box[0])
    return boxes_list, labels

In [30]:
def save_image(image, name: str) -> None:
    image_path = aug_dir / 'images' / f'{name}.jpg'
    cv2.imwrite(str(image_path), image)

In [31]:
def write_txt(boxes, name: str) -> None:
    txt_file = aug_dir / 'labels' / f'{name}.txt'
    with open(txt_file, 'w') as new_file:
        for row in boxes:
            new_file.write(row)

In [32]:
def save_label(boxes, name: str) -> None:
    rows = []
    for box in boxes:
        row = ' '.join([f'{x}' for x in box])
        row = '0 ' + row + '\n'
        rows.append(row)
    write_txt(rows, name)
        

In [33]:
def show_image(image, mask):
    if len(mask) > 0:
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(25,25))
        axes[0].imshow(image)
        #axes[1].imshow(mask[..., 0])
        plt.show

In [34]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White

def visualize_bbox(img, bbox, class_name, color=(250,0,0), thickness=1):
    """Visualizes a single bounding box on the image"""
    print(img.shape)
    x_c, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_c*256-(w*256)), int(x_c*256 + (w*256)), int(y_min*256-(h*256)), int(y_min*256 + (h*256))
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.25, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img

def visualize(image, bboxes, category_ids=['0'], category_id_to_name={'0': 'person'}):
    img = image.copy()
    
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [35]:
def get_size(image_file):    
    image = Image.open(image_file)    
    return image.size

In [36]:
def start_aug():
    errors_box = 0
    for numer, label_file in enumerate(labels_list):
        short_name = label_file.stem
        image_name = location[location['short_name']==short_name]['ID_img'].values[0]
        image_path = location_image / image_name
        image = get_image(image_path)
        bboxes, class_labels = get_boxes(label_file)
        size = get_size(image_path)
        
        for num in range(person_num):
            name = f'{short_name}p{num}'
            try:
                new_image, new_bboxes = transformer_person(image, bboxes, class_labels, size)
                #print(bboxes, new_bboxes)
                category_ids = ['0']* len(new_bboxes)
                #visualize(new_image, new_bboxes, category_ids)
                save_image(new_image, name)
                save_label(new_bboxes, name)
            except:
                errors_box += 1
            
        for num in range(area_num):
            name = f'{short_name}a{num}'
            try:
                new_image, new_bboxes = transformer_area(image, bboxes, class_labels, size)
                category_ids = ['0'] * len(new_bboxes)
                #visualize(new_image, new_bboxes, category_ids)
                save_image(new_image, name)
                if len(new_bboxes) > 0:
                    save_label(new_bboxes, name)
            except:
                errors_box += 1
            
        print(numer)
    print(errors_box)

In [37]:
start_aug()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
0


In [38]:
!ls

augumentation  forest_baseline.ipynb		README.md  ya.py
convert.ipynb  forest_omsk_train_model_2.ipynb	train	   yolov5
dataset        poetry.lock			train.csv
datasets       pyproject.toml			venv


In [ ]:
!rm -r train

In [39]:
!ls

augumentation  forest_baseline.ipynb		README.md  ya.py
convert.ipynb  forest_omsk_train_model_2.ipynb	train	   yolov5
dataset        poetry.lock			train.csv
datasets       pyproject.toml			venv


In [40]:
!zip -r dataset.zip augumentation

/bin/bash: строка 1: zip: команда не найдена


In [ ]:
!ls

In [ ]:
!rm -r augumentation

In [ ]:
!mkdir dataset
!mv dataset.zip dataset/dataset.zip

In [ ]:
!ls dataset

In [ ]:
!cp -r dataset /content/drive/MyDrive/colab_export/omsk_forest/import/

In [ ]:
%cd dataset

In [ ]:
!du -sh *

In [ ]:
!ls /content/drive/MyDrive/colab_export/omsk_forest/import/dataset

In [ ]:
!rm dataset.zip

In [ ]:
!ls

In [ ]:
!rm -r augu

In [ ]:
!mkdir train

In [ ]:
!ls

### Загрузка готового дасета для обучения

In [ ]:
!ls /content/drive/MyDrive/colab_export/omsk_forest/import/

In [ ]:
!cp /content/drive/MyDrive/colab_export/omsk_forest/import/dataset/dataset.zip dataset.zip

In [ ]:
!unzip dataset.zip

# Создание набора тестовых и тренироваочных даных


In [ ]:
crop_image = aug_dir / 'images'
crop_labels = aug_dir / 'labels'

In [ ]:
crop_image_list = get_files(crop_image, 'jpg')
crop_labels_list = get_files(crop_labels)


In [ ]:
crop_labels_list = [Path(label_path).stem for label_path in crop_labels_list]
#crop_labels_list

In [ ]:
dataset_df = pd.DataFrame(columns=['image', 'label', 'image_path', 'label_path'])

In [ ]:
dataset_df['image_path'] = crop_image_list

In [ ]:
dataset_df.head(3)

In [ ]:
def path_to_name(row: pd.Series) -> pd.Series:
    image = Path(row['image_path'])
    row['image'] = image.name
    if image.stem in crop_labels_list:
        row['label'] = f'{image.stem}.txt'
        row['label_path'] = aug_dir / 'labels' /row['label']
    return row

In [ ]:
dataset_df.apply(path_to_name, axis=1)

In [ ]:
dataset_df.head(3)

In [ ]:
label_df = dataset_df[dataset_df['label'].notna()].copy()
non_label_df = dataset_df[dataset_df['label'].isna()].copy()

In [ ]:
label_df.head(2)

In [ ]:
dataset_df.shape[0] - label_df.shape[0] - non_label_df.shape[0]

In [ ]:
def train_validate_test_split(df: pd.DataFrame, train_percent: float = 0.6, validate_percent: float = 0.4, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [ ]:
train_df_1, valid_df_1, _ = train_validate_test_split(label_df.reset_index(drop=True))

In [ ]:
train_df_2, valid_df_2, _ = train_validate_test_split(non_label_df.reset_index(drop=True))

In [ ]:
train_df = pd.concat([train_df_1, train_df_2])
valid_df = pd.concat([valid_df_1, valid_df_2])

In [ ]:
dataset_df.shape, valid_df.shape, train_df.shape

In [ ]:
11595+17393

In [ ]:
valid_df

In [ ]:
import shutil
import os

def move_file(short_name: str, target_dir: str) -> None:
    image_file = aug_dir / 'images' / f'{short_name}.jpg'
    new_image = aug_dir / 'images' / target_dir / f'{short_name}.jpg'
    shutil.move(image_file, new_image)
    
    label_file = aug_dir / 'labels' / f'{short_name}.txt'
    new_label = aug_dir / 'labels' / target_dir / f'{short_name}.txt'
    print(label_file)
    print(new_label)
    print(label_file.is_file())
    if label_file.is_file():
        print('ok')
        shutil.move(label_file, new_label)


In [ ]:
def move_handler(row, target_dir):
    name = Path(row['image_path']).stem
    move_file(str(name), target_dir)
    return row

In [ ]:
valid_df.apply(move_handler, axis=1, args=('test',))

In [ ]:
train_df.apply(move_handler, axis=1, args=('train',))

In [ ]:
!ls augumentation/images

# Обучение модели

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

In [ ]:
!ls data

In [ ]:
!cp /content/drive/MyDrive/colab_export/omsk_forest/import/forest1.yaml data/forest.yaml

In [ ]:
!mkdir train train/images train/labels

In [ ]:
!mv /content/augumentation/images/train train/images
!mv /content/augumentation/images/test train/images
!mv /content/augumentation/labels/train/ train/labels
!mv /content/augumentation/labels/test train/labels

In [ ]:
!ls train/images/train

In [ ]:
!cp /content/drive/MyDrive/colab_export/omsk_forest/export/model_4/best.pt best.pt

In [ ]:
!ls runs/train

In [ ]:
!python train.py --batch 16 --epochs 5 --data forest.yaml --weights best.pt
!cp -r runs/train/exp /content/drive/MyDrive/colab_export/omsk_forest/export/model_4/

In [ ]:
!rm best.pt


In [ ]:
!cp /content/drive/MyDrive/colab_export/omsk_forest/export/model_4/exp/weights/best.pt best.pt

In [ ]:
!python train.py --batch 16 --epochs 6 --data forest.yaml --weights best.pt
!cp -r runs/train/exp2 /content/drive/MyDrive/colab_export/omsk_forest/export/model_4/